In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Classe das partículas e parede

In [4]:
class Particula:
    def __init__(self, raio: float, massa: float, posicao: float, velocidade: float) -> None:
        self.massa = massa
        self.raio = raio
        
        self.posicao_x = posicao[0]
        self.posicao_y = posicao[1]

        self.velocidade_x = velocidade[0]
        self.velocidade_y = velocidade[1]

In [ ]:
paredes = []

Testagem

In [28]:
p1 = Particula(10,5,[1,2],[2,1])